In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [87]:
import os
import datetime
import json
import requests

import pandas as pd
import tqdm

In [17]:
API_KEY = os.environ['API_KEY']
TEAM_ID = os.environ['TEAM_ID']
MEMBER_ID = os.environ['MEMBER_ID']

# Load Parameters

In [93]:
time_sheet_path = 'input/time_sheet_20230317-20230331.xlsx'
# time_sheet_path = 'input/time_sheet_20230401-20230422.xlsx'

# Time Tracking 

In [94]:
time_sheet_df = pd.read_excel(time_sheet_path)
time_sheet_df

,url,task,2023-03-17,2023-03-18,2023-03-19,2023-03-20,2023-03-21,2023-03-22,2023-03-23,2023-03-24,2023-03-25,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31
0,https://app.clickup.com/t/2egpdre,MRT - Word Cloud,NaN,NaN,NaN,7.5,7.5,4.0,NaN,NaN,7.5,NaN,NaN,NaN,NaN,NaN,NaN
1,https://app.clickup.com/t/2egpbkx,MRT - Demand Forecasting,NaN,NaN,NaN,NaN,NaN,3.5,NaN,3.5,NaN,NaN,NaN,7.5,7.5,3.5,3.5
2,https://app.clickup.com/t/2qvp4qe,GSB - Product Rec,NaN,NaN,NaN,NaN,NaN,NaN,7.5,4.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0


In [95]:
time_spent_df = time_sheet_df.drop(columns=['task']).melt(id_vars=['url'], var_name='date', value_name='time_spent').dropna()

# Extract last 6 characters of url
time_spent_df['task_id'] = time_spent_df['url'].str.extract(r'https://app.clickup.com/t/(\w+)', expand=False)
time_spent_df['date'] = pd.to_datetime(time_spent_df['date'], format='%Y-%m-%d')
time_spent_df

,url,date,time_spent,task_id
9,https://app.clickup.com/t/2egpdre,2023-03-20,7.5,2egpdre
12,https://app.clickup.com/t/2egpdre,2023-03-21,7.5,2egpdre
15,https://app.clickup.com/t/2egpdre,2023-03-22,4.0,2egpdre
16,https://app.clickup.com/t/2egpbkx,2023-03-22,3.5,2egpbkx
20,https://app.clickup.com/t/2qvp4qe,2023-03-23,7.5,2qvp4qe
22,https://app.clickup.com/t/2egpbkx,2023-03-24,3.5,2egpbkx
23,https://app.clickup.com/t/2qvp4qe,2023-03-24,4.0,2qvp4qe
24,https://app.clickup.com/t/2egpdre,2023-03-25,7.5,2egpdre
34,https://app.clickup.com/t/2egpbkx,2023-03-28,7.5,2egpbkx
37,https://app.clickup.com/t/2egpbkx,2023-03-29,7.5,2egpbkx


In [96]:
def create_time_entry(task_id: str, dt: datetime.datetime, duration: float) -> dict:
    url = f'https://api.clickup.com/api/v2/team/{TEAM_ID}/time_entries'

    dt = dt.replace(hour=9)
    payload = {
        'description': 'time entry',
        'tags': [],
        'start': int(dt.timestamp() * 1000),
        'billable': False,
        'duration': int(duration * 60 * 60 * 1000),
        'assignee': int(MEMBER_ID),
        'tid': task_id
    }

    headers = {
        'Content-Type': 'application/json',
        'Authorization': API_KEY
    }

    response = requests.post(url, json=payload, headers=headers)
    data = response.json()
    return data

In [97]:
for _, time_entry in tqdm.tqdm(time_spent_df.iterrows(), total=len(time_spent_df)):
    print(f'Creating time entry for {time_entry["task_id"]} on {time_entry["date"]} for {time_entry["time_spent"]} hours')
    create_time_entry(time_entry['task_id'], time_entry['date'].to_pydatetime(), time_entry['time_spent'])

  0%|          | 0/14 [00:00<?, ?it/s]

Creating time entry for 2egpdre on 2023-03-20 00:00:00 for 7.5 hours


  7%|▋         | 1/14 [00:00<00:10,  1.23it/s]

Creating time entry for 2egpdre on 2023-03-21 00:00:00 for 7.5 hours


 14%|█▍        | 2/14 [00:01<00:09,  1.28it/s]

...

 86%|████████▌ | 12/14 [00:10<00:01,  1.05it/s]

Creating time entry for 2egpbkx on 2023-03-31 00:00:00 for 3.5 hours


 93%|█████████▎| 13/14 [00:11<00:01,  1.06s/it]

Creating time entry for 2qvp4qe on 2023-03-31 00:00:00 for 4.0 hours


100%|██████████| 14/14 [00:12<00:00,  1.11it/s]


In [89]:
time_spent_df['date'].iloc[0].to_pydatetime()

datetime.datetime(2023, 4, 3, 0, 0)

In [ ]:



url = "https://api.clickup.com/api/v2/team/" + TEAM_ID + "/time_entries"

query = {
  # "custom_task_ids": "true",
  # "team_id": "123"
}

payload = {
  "description": "time entry",
  "tags": [
    # {
    #   "name": "name of tag",
    #   "tag_bg": "#BF55EC",
    #   "tag_fg": "#FFFFFF"
    # }
  ],
  "start": start,
  "billable": False,
  "duration": int(duration),
  "assignee": int(MEMBER_ID),
  "tid": task_id
}

headers = {
  "Content-Type": "application/json",
  "Authorization": API_KEY
}

response = requests.post(url, json=payload, headers=headers, params=query)

data = response.json()
print(data)